In [1]:
import pandas as pd
import numpy as np
import os
os.listdir()

['.ipynb_checkpoints',
 'Decision Tree Scratch.ipynb',
 'diabetes.csv',
 'tennis weather.csv']

In [2]:
data = pd.read_csv('tennis weather.csv')

In [3]:
data

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [4]:
data.dtypes

outlook     object
temp        object
humidity    object
windy         bool
play        object
dtype: object

In [5]:
data['windy'] = data['windy'].astype('object')
data['windy'] = data['windy'].map({False:'weak',True:'strong'})

In [6]:
data.head()

,outlook,temp,humidity,windy,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes


In [7]:
def cal_entropy(df_label):
    classes,value_counts  = np.unique(df_label,return_counts=True)
    entropy=0
    for cls in range(len(classes)):
        entropy += -1*(value_counts[cls]/np.sum(value_counts))*np.log2(value_counts[cls]/np.sum(value_counts))
    return entropy

In [8]:
def cal_info_gain(dataset,feature,label):
    
    dataset_entropy = cal_entropy(dataset[label])
    
    feature_classes,value_counts = np.unique(dataset[feature],return_counts=True)
    
    weighted_feature_entropy = 0
    for i in range(len(feature_classes)):
        weighted_feature_entropy += (value_counts[i]/np.sum(value_counts))*cal_entropy(dataset.iloc[np.where(dataset[feature]==feature_classes[i])][label])
    
    info_gain = dataset_entropy - weighted_feature_entropy
    
    return info_gain

In [9]:
info_gain = []
for features in ['humidity','temp','windy','outlook']:
    info_gain.append(cal_info_gain(data,features,'play'))
info_gain

[0.15183550136234159,
 0.02922256565895487,
 0.04812703040826949,
 0.24674981977443933]

entropy = (-6/8)*np.log2(6/8)+(-2/8)*np.log2(2/8)

weighted entropy = (8/14)*((-6/8)*np.log2(6/8)+(-2/8)*np.log2(2/8)) + (6/14)*((-3/6)*np.log2(3/6)+(-3/6)*np.log2(3/6))

When do we end the decision tree?

1. when the label value, is same on every row of data subset
2. if there is no more features left for splitting
3. when there is no more data left for splitting

In [10]:
def create_decision_tree(dataset,df,features,label,parent):
    
    datum = np.unique(df[label],return_counts=True)
    unique_data = np.unique(dataset[label])
    
    if len(unique_data)<=1:
        return unique_data[0]
    
    elif len(dataset)==0:
        return unique_data[np.argmax(datum[1])]
    
    elif len(features) == 0:
        return parent
    else:
        parent = unique_data[np.argmax(datum[1])]
        
        item_values = [cal_info_gain(dataset,feature,label) for feature in features]
        
        optimum_feature = features[np.argmax(item_values)]
        
        decision_tree = {optimum_feature:{}}
        
        features = [i for i in features if i!= optimum_feature]
        
        
        for value in np.unique(dataset[optimum_feature]):
            
            min_data = dataset.where(dataset[optimum_feature]==value).dropna()
            
            min_tree = create_decision_tree(min_data,df,features,label,parent)

            decision_tree[optimum_feature][value]= min_tree
            
        return(decision_tree)

In [11]:
def predict(test_data,decision_tree):
    
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        decision_tree = decision_tree[nodes][value]
        
        prediction = 'no'
        if type(decision_tree) is dict:
            
            prediction = predict(test_data,decision_tree)
        else:
            prediction = decision_tree
            break;
    return prediction

In [12]:
features = data.columns[:-1]
label = 'play'
parent = None
features

Index(['outlook', 'temp', 'humidity', 'windy'], dtype='object')

In [14]:
decision_tree = create_decision_tree(data,data,features,label,parent)

In [15]:
decision_tree

{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {'strong': 'no', 'weak': 'yes'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}

# Prediction

In [16]:
sample_data = {'outlook':'rainy','temp':'mild','humidity':'high','windy':'weak'}
test_data = pd.Series(sample_data)
prediction = predict(test_data,decision_tree)
prediction

'yes'